In [ ]:
# Load packages
import numpy as np
import pandas as pd
from google.colab import files
import re
from nltk.corpus import stopwords
from nltk import word_tokenize
from bs4 import BeautifulSoup
from sklearn.model_selection import train_test_split
from gensim.models import Word2Vec, KeyedVectors
from gensim.scripts.glove2word2vec import glove2word2vec
import tensorflow as tf
from tensorflow.keras.layers import Dense, Embedding, GRU, SpatialDropout1D, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import classification_report
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Import the cleaned dataset with the glove embeddings
df = pd.read_csv('/content/customer_complaints_clean_vF.csv')
df.head()

,product,narrative
0,credit_card,purchase order day shipping amount receive pro...
1,credit_card,forwarded message date tue subject please inve...
2,retail_banking,forwarded message cc sent friday pdt subject f...
3,credit_reporting,payment history missing credit report speciali...
4,credit_reporting,payment history missing credit report made mis...


In [ ]:
import urllib.request

url = 'https://nlp.stanford.edu/data/glove.6B.zip'
urllib.request.urlretrieve(url, 'glove.6B.zip')


('glove.6B.zip', <http.client.HTTPMessage at 0x7f8eb8526a30>)

In [ ]:
import zipfile

with zipfile.ZipFile('glove.6B.zip', 'r') as zip_ref:
    zip_ref.extractall('glove')


In [ ]:
def load_embedding(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        lines = f.readlines()[1:]
        embedding_dict = {}
        for line in lines:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embedding_dict[word] = coefs
    return embedding_dict

In [ ]:
embedding_dict = load_embedding('glove/glove.6B.100d.txt')

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Separate target and input data
X = df['narrative'].astype(str).tolist()
Y = df['product'].astype(str).tolist()

In [ ]:
print(df.dtypes)


product      object
narrative    object
dtype: object


In [ ]:
from sklearn.preprocessing import LabelEncoder
# Convert target labels to numerical labels
label_encoder = LabelEncoder()
Y = label_encoder.fit_transform(Y)
from keras.utils import to_categorical

# Convert target data to one-hot encoded vectors
Y = to_categorical(Y, num_classes=5)

# Remove extra dimension from Y
Y = Y.reshape(-1, 5)

In [ ]:
Y.shape

(29202, 5)

In [ ]:
# Convert X to list of strings
X_str = [str(sentence) for sentence in X]

# Tokenize input data
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_str)
X = tokenizer.texts_to_sequences(X_str)
X = pad_sequences(X, maxlen=200)


In [ ]:
# Create embedding matrix for input vocabulary
word_index = tokenizer.word_index
embedding_matrix = np.zeros((len(word_index) + 1, 100))
for word, i in word_index.items():
    embedding_vector = embedding_dict.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [ ]:
# Split data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)


In [ ]:
from sklearn.utils.class_weight import compute_class_weight

# Convert one-hot encoded labels to class labels
Y_train_labels = np.argmax(Y_train, axis=1)

# Compute class weights
class_weights = compute_class_weight('balanced', classes=np.unique(Y_train_labels), y=Y_train_labels)

# Convert class weights to dictionary
class_weights_dict = dict(enumerate(class_weights))

In [ ]:
#Y = np.squeeze(Y)

In [ ]:
# Define the number of words to consider
num_words = 5000
maxlen = max([len(seq) for seq in X])

In [ ]:
# Build LSTM model
from keras.layers import LSTM
from keras.models import Sequential

model = Sequential()
model.add(Embedding(input_dim=num_words, output_dim=32, input_length=maxlen))
model.add(LSTM(units=64))
model.add(Dense(5, activation='softmax'))


model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
history = model.fit(X_train, Y_train, epochs=10, batch_size=128, validation_split=0.2, class_weight=class_weights_dict, callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])


Epoch 1/10
146/146 [==============================] - 62s 381ms/step - loss: 1.2618 - accuracy: 0.4924 - val_loss: 0.9479 - val_accuracy: 0.6242
Epoch 2/10
146/146 [==============================] - 52s 360ms/step - loss: 0.7731 - accuracy: 0.6575 - val_loss: 0.7981 - val_accuracy: 0.6891
Epoch 3/10
146/146 [==============================] - 48s 330ms/step - loss: 0.6574 - accuracy: 0.7422 - val_loss: 0.7621 - val_accuracy: 0.7451
Epoch 4/10
146/146 [==============================] - 47s 319ms/step - loss: 0.5039 - accuracy: 0.8038 - val_loss: 0.6775 - val_accuracy: 0.7723
Epoch 5/10
146/146 [==============================] - 47s 323ms/step - loss: 0.4177 - accuracy: 0.8418 - val_loss: 0.6227 - val_accuracy: 0.7933
Epoch 6/10
146/146 [==============================] - 48s 331ms/step - loss: 0.3687 - accuracy: 0.8600 - val_loss: 0.6066 - val_accuracy: 0.7995
Epoch 7/10
146/146 [==============================] - 47s 322ms/step - loss: 0.3490 - accuracy: 0.8581 - val_loss: 0.7912 - val_ac

In [ ]:
scores = model.evaluate(X_test, Y_test)
print(f"Test Loss: {scores[0]:.4f}")
print(f"Test Accuracy: {scores[1]*100:.2f}%")

183/183 [==============================] - 6s 32ms/step - loss: 0.6533 - accuracy: 0.7860
Test Loss: 0.6533
Test Accuracy: 78.60%


In [ ]:
Y_test = np.argmax(Y_test, axis=1)
Y_pred = model.predict(X_test)
Y_pred = np.argmax(Y_pred, axis=1)
print(classification_report(Y_test, Y_pred))

183/183 [==============================] - 8s 37ms/step
              precision    recall  f1-score   support

           0       0.65      0.73      0.69       818
           1       0.89      0.79      0.84      2391
           2       0.71      0.79      0.75      1057
           3       0.81      0.81      0.81       964
           4       0.76      0.80      0.78       611

    accuracy                           0.79      5841
   macro avg       0.76      0.78      0.77      5841
weighted avg       0.79      0.79      0.79      5841

